In [1]:
import tensorflow as tf

#Block of layers with bool for if grad = true
#Grad = not true, no flatten and FCL

class GroupedLayer(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, final_output):
        super(GroupedLayer, self).__init__()
        self.hidden_1 = tf.keras.layers.Conv2D(filters, kernel_size, strides=(1,1), padding="same", activation='ReLU')
        self.hidden_2 = tf.keras.layers.Conv2D(filters, kernel_size, strides=(1,1), padding="same", activation='ReLU')
        self.hidden_output = tf.keras.layers.Dense(units=final_output, activation='ReLU')
        
    def call(self, inputs):
        inputs = self.hidden_1(inputs)
        inputs = self.hidden_2(inputs)
        return inputs
    
    def call_output(self, inputs):
          return self.hidden_output(tf.keras.layers.Flatten()(self.call(inputs)))

class GroupedLayerInput(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, final_output):
        super(GroupedLayerInput, self).__init__()
        self.input_block = tf.keras.Sequential(
            [
                tf.keras.Input(shape=(32, 32, 3)),
                tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same", activation="ReLU"),
                tf.keras.layers.MaxPooling2D(pool_size=2),

            ],
            name="input",
        )
        self.hidden = GroupedLayer(64, (3,3), 6)
        
    def call(self, inputs):
        inputs = self.input_block(inputs)
        inputs = self.hidden.call(inputs)
        return inputs
    
    def call_output(self, inputs):
          return self.hidden(tf.keras.layers.Flatten()(self.call(inputs)))

grouped_block = GroupedLayer(64,3,10)

In [11]:
loss_tracker = tf.keras.metrics.Mean(name="loss")
class ForcedLearnerClass(tf.keras.Model):
    
    def __init__(self, output_size):
        super(ForcedLearnerClass, self).__init__()
        self.input_block = GroupedLayerInput(filters=16,kernel_size=(3,3), final_output=output_size)
        
        self.training_block_1 = [GroupedLayer(16, (3,3), final_output=output_size) for _ in range(3)]
        self.training_block_2 = [GroupedLayer(32, (3,3), final_output=output_size) for _ in range(3)]
        self.training_block_3 = [GroupedLayer(64, (3,3), final_output=output_size) for _ in range(3)]
    
    def compile(self, optimizer, loss_fn):
        super(ForcedLearnerClass, self).compile()
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        
    def call(self, x):
        x = self.input_block(x)
        for i in range(3):
            x = self.training_block_1[i].call(x)
        for i in range(3):
            x = self.training_block_2[i].call(x)
        for i in range(3):
            x = self.training_block_3[i].call(x)
        return x
        
    def train_step(self, data):
        pred_output, y = data
        
        with tf.GradientTape(persistent=True) as tape:
            pred_output_back_input = self.input_block.call_output(pred_output)
            pred_output = self.input_block.call(pred_output)
            
            pred_output_back_1 = []
            for i in range(3):
                pred_output_back_1.append(self.training_block_1[i].call_output(pred_output))
                pred_output = self.training_block_1[i].call(pred_output)
                
            pred_output_back_2 = []
            for i in range(3):
                pred_output_back_2.append(self.training_block_2[i].call_output(pred_output))
                pred_output = self.training_block_2[i].call(pred_output)
                
            pred_output_back_3 = []
            for i in range(3):
                pred_output_back_3.append(self.training_block_3[i].call_output(pred_output))
                pred_output = self.training_block_3[i].call(pred_output)
                
            loss_input = self.loss_fn(y, pred_output_back_input)
            loss_1 = []
            loss_2 = []
            loss_3 = []
            
            for i in range(3):
                loss_1[i].append(self.loss_fn(y, pred_output_back_1[i]))
                loss_2[i].append(self.loss_fn(y, pred_output_back_2[i]))
                loss_3[i].append(self.loss_fn(y, pred_output_back_3[i]))
                
        #mae_metric = keras.metrics.MeanAbsoluteError(name="mae")
        gradients_input = tape.gradient(loss_input, self.input_block.trainable_variables)
        gradients_1 = []
        gradients_2 = []
        gradients_3 = []
        
        for i in range(3):
            gradients_1.append(tape.gradient(loss_1[i], self.training_block_1[i].trainable_variables))
            gradients_2.append(tape.gradient(loss_2[i], self.training_block_2[i].trainable_variables))
            gradients_3.append(tape.gradient(loss_3[i], self.training_block_3[i].trainable_variables))
        
        #Update weights
        self.optimizer.apply_gradients(zip(gradients_input, self.input_block.trainable_variables))
        for i in range(3):
            self.optimizer.apply_gradients(zip(gradients_1[i], self.training_block_1[i].trainable_variables))
            self.optimizer.apply_gradients(zip(gradients_2[i], self.training_block_2[i].trainable_variables))
            self.optimizer.apply_gradients(zip(gradients_3[i], self.training_block_3[i].trainable_variables))
         
        #Compute our own metrics
        loss_tracker.update_state(loss_4)
        #mae_metric.update_state(y, y_pred)
        #"mae": mae_metric.result()
        return {"loss": loss_tracker.result()}

In [3]:
import tensorflow as tf
training_block_1 = tf.keras.Sequential(
            [
                GroupedLayer(64, (64,64), 6) for _ in range(2)
            ]
        )

training_block_1.build((16,224,224,3))

training_block_1.summary()
training_block_1.trainable_variables
training_block_1.compile()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 grouped_layer_1 (GroupedLay  (16, 224, 224, 64)       17563776  
 er)                                                             
                                                                 
 grouped_layer_2 (GroupedLay  (16, 224, 224, 64)       33554560  
 er)                                                             
                                                                 
Total params: 51,118,336
Trainable params: 51,118,336
Non-trainable params: 0
_________________________________________________________________


In [4]:
network_block = GroupedLayerInput(2, (3,3), 6)

network_block.build((16,224,224,3))

In [12]:
forced_learner = ForcedLearnerClass(output_size=10)
forced_learner.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=tf.keras.losses.Huber(),
)

In [13]:
batch_size = 10
(x_train, Y_train), (x_test, Y_test) = tf.keras.datasets.cifar10.load_data()

In [14]:
import numpy as np
all_digits = np.concatenate([x_train, x_test])
all_output = np.concatenate([Y_train, Y_test])
all_output_one_hot = tf.one_hot(np.squeeze(all_output), 10)
all_digits = all_digits.astype("float32") / 255.0

In [15]:
all_digits = np.reshape(all_digits, (-1, 32, 32, 3))
dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_output_one_hot))
dataset = dataset.shuffle(buffer_size=1024)
dataset = dataset.batch(batch_size).prefetch(1)

In [16]:
DATASET_SIZE=60000
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

full_dataset = dataset
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)

In [17]:
history = forced_learner.fit(train_dataset, epochs=10)

Epoch 1/10


NameError: in user code:

    File "/home/jack/ml/my_env/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/jack/ml/my_env/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/jack/ml/my_env/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/tmp/ipykernel_132726/1120599091.py", line 60, in train_step
        gradients_input = tape.gradient(loss_input, self.input_block.trainable_variables)

    NameError: name 'tape' is not defined


In [ ]:
model_temp = tf.keras.applications.ResNet50V2(weights=None, input_shape=(32,32,3), classes=10)

opt = tf.keras.optimizers.Nadam(learning_rate=3e-4)

callback = [tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=7)]
model_temp.compile(loss="Huber", optimizer=opt, metrics=['mse', 'mae', 'mape', 'accuracy'])
history = model_temp.fit(train_dataset, batch_size=16, epochs=100, callbacks=[callback],
                    validation_data=(val_dataset))

In [ ]:
np.__version__

In [ ]:
all_output_one_hot[0]

In [ ]:
input_block = GroupedLayerInput(filters=64,kernel_size=(3,3), final_output=6)
training_block_1 = [GroupedLayer(64, (64,64), 6) for _ in range(2)]
training_block_2 = [GroupedLayer(64, (64,64), 6) for _ in range(2)]
training_block_3 = [GroupedLayer(64, (64,64), 6) for _ in range(2)]
training_block_4 = [GroupedLayer(64, (64,64), 6) for _ in range(2)]

In [ ]:
train_dataset.take(1)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()
epochs = 2

# Iterate over epochs.
for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
           
        
            #Compute Loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses)  # Add KLD regularization loss

        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))

        loss_metric(loss)

        if step % 100 == 0:
            print("step %d: mean loss = %.4f" % (step, loss_metric.result()))
